


A convolutional neural network model for survival prediction based on prognosis-related cascaded Wx feature selection

https://www.laboratoryinvestigation.org/article/S0023-6837(22)00114-3/fulltext

https://github.com/wangwangCCChen/CNN-Cox

https://github.com/ucsffrancislab/CNN-Cox


In [1]:
#from Trainmodel import traindcnncoxmodel,traincnncoxmodel
#from utils import avgcindex

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import math
import numpy as np
import pandas as pd
#from model import cnncox,dcnncox,nncox
from earlystoping import MyCallback
from utils import nll,avgcindex,setup_seed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold,train_test_split
from lifelines.utils import concordance_index
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,Dropout,Input,Flatten,concatenate
from tensorflow.keras.regularizers import l2

2024-04-18 11:54:57.195673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



GENE_2_counts_normalized.subject_unique.metadata.all.tsv

was created from the REdiscoverTE output converted to a tsv.

The metadata came from Eduardo's initial gbm_train/gbm_test data files.


In [3]:

cancer_name='TCGA'
Cindex=[]

current_path='../data/our/'
output_path=current_path+'/GENE_2_counts_normalized.subject_unique.metadata.all.tsv'
save_path="../save model/our/my2DCNNCox/"
os.makedirs(save_path,exist_ok=True)


In [4]:

seed=42

setup_seed(seed)

df_exp=pd.read_csv(output_path,index_col=0,sep="\t",low_memory=False)


In [5]:
df_exp.head()

,sex,IDH,x1p19q,TERT,MGMT,Age,Survival_months,Vital_status,EGFR_re,CDKN2A_re,...,ENSG00000284649,ENSG00000284671,ENSG00000284680,ENSG00000284684,ENSG00000284691,ENSG00000284701,ENSG00000284723,"ENSG00000284733,ENSG00000284662",ENSG00000284741,NA
IID,,,,,,,,,,,,,,,,,,,,,
02-0047,male,WT,non-codel,NaN,Unmethylated,78,14.718995,1,Wildtype,Deleted,...,7.766,9.993,0,0.0,47.574,0,1,0.0,0.0,569.221
02-0055,female,WT,non-codel,NaN,Unmethylated,62,2.496972,1,Amplified,Wildtype,...,10.742,0.000,0,0.0,25.980,0,0,0.0,0.0,491.114
02-2485,male,WT,non-codel,Mutant,Unmethylated,53,15.441803,0,Amplified,Wildtype,...,5.379,0.000,0,0.0,69.000,0,1,0.0,0.0,717.150
02-2486,male,WT,non-codel,NaN,Unmethylated,64,16.197466,0,Amplified,Deleted,...,2.037,4.746,0,0.0,49.948,0,0,0.0,0.0,274.183
06-0125,female,WT,non-codel,Mutant,Methylated,63,47.573895,1,Amplified,Deleted,...,8.879,4.237,0,0.0,122.020,0,1,4.0,0.0,657.634


In [6]:
df_exp.drop(columns=['IDH','x1p19q','TERT','MGMT'],inplace=True)
df_exp.head()


# EGFR_re, CDKN2A_re ?

# what is the column "NA" from?

,sex,Age,Survival_months,Vital_status,EGFR_re,CDKN2A_re,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,...,ENSG00000284649,ENSG00000284671,ENSG00000284680,ENSG00000284684,ENSG00000284691,ENSG00000284701,ENSG00000284723,"ENSG00000284733,ENSG00000284662",ENSG00000284741,NA
IID,,,,,,,,,,,,,,,,,,,,,
02-0047,male,78,14.718995,1,Wildtype,Deleted,2350.460,1.000,633.000,439.035,...,7.766,9.993,0,0.0,47.574,0,1,0.0,0.0,569.221
02-0055,female,62,2.496972,1,Amplified,Wildtype,2893.011,825.001,2382.838,569.035,...,10.742,0.000,0,0.0,25.980,0,0,0.0,0.0,491.114
02-2485,male,53,15.441803,0,Amplified,Wildtype,4940.435,5.000,1128.869,695.463,...,5.379,0.000,0,0.0,69.000,0,1,0.0,0.0,717.150
02-2486,male,64,16.197466,0,Amplified,Deleted,4338.707,7.000,1089.000,588.688,...,2.037,4.746,0,0.0,49.948,0,0,0.0,0.0,274.183
06-0125,female,63,47.573895,1,Amplified,Deleted,3621.526,6.000,1226.225,632.096,...,8.879,4.237,0,0.0,122.020,0,1,4.0,0.0,657.634


In [7]:
#df_exp.fillna(0,inplace=True)

if 'sex' in df_exp.columns:
    df_exp['sex'].replace(to_replace={'female':0, 'male':1}, inplace=True)
if 'MGMT' in df_exp.columns:
    df_exp['MGMT'].replace(to_replace={'Unmethylated':0, 'Methylated':1}, inplace=True)
if 'EGFR_re' in df_exp.columns:
    df_exp['EGFR_re'].replace(to_replace={"Wildtype":0, "Amplified":1}, inplace=True)
if 'CDKN2A_re' in df_exp.columns:
    df_exp['CDKN2A_re'].replace(to_replace={"Wildtype":0, "Amplified":1, "Deleted":2}, inplace=True)

df_exp.head()


,sex,Age,Survival_months,Vital_status,EGFR_re,CDKN2A_re,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,...,ENSG00000284649,ENSG00000284671,ENSG00000284680,ENSG00000284684,ENSG00000284691,ENSG00000284701,ENSG00000284723,"ENSG00000284733,ENSG00000284662",ENSG00000284741,NA
IID,,,,,,,,,,,,,,,,,,,,,
02-0047,1,78,14.718995,1,0,2,2350.460,1.000,633.000,439.035,...,7.766,9.993,0,0.0,47.574,0,1,0.0,0.0,569.221
02-0055,0,62,2.496972,1,1,0,2893.011,825.001,2382.838,569.035,...,10.742,0.000,0,0.0,25.980,0,0,0.0,0.0,491.114
02-2485,1,53,15.441803,0,1,0,4940.435,5.000,1128.869,695.463,...,5.379,0.000,0,0.0,69.000,0,1,0.0,0.0,717.150
02-2486,1,64,16.197466,0,1,2,4338.707,7.000,1089.000,588.688,...,2.037,4.746,0,0.0,49.948,0,0,0.0,0.0,274.183
06-0125,0,63,47.573895,1,1,2,3621.526,6.000,1226.225,632.096,...,8.879,4.237,0,0.0,122.020,0,1,4.0,0.0,657.634


In [8]:
gene_exp=df_exp.dropna(axis=0,subset = ["Vital_status", "Survival_months"])
E=np.array(gene_exp.pop("Vital_status"))
Y=np.array(gene_exp.pop("Survival_months"))

In [9]:
gene_exp.head()

,sex,Age,EGFR_re,CDKN2A_re,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,...,ENSG00000284649,ENSG00000284671,ENSG00000284680,ENSG00000284684,ENSG00000284691,ENSG00000284701,ENSG00000284723,"ENSG00000284733,ENSG00000284662",ENSG00000284741,NA
IID,,,,,,,,,,,,,,,,,,,,,
02-0047,1,78,0,2,2350.460,1.000,633.000,439.035,304.000,534.288,...,7.766,9.993,0,0.0,47.574,0,1,0.0,0.0,569.221
02-0055,0,62,1,0,2893.011,825.001,2382.838,569.035,497.997,1123.424,...,10.742,0.000,0,0.0,25.980,0,0,0.0,0.0,491.114
02-2485,1,53,1,0,4940.435,5.000,1128.869,695.463,743.572,503.997,...,5.379,0.000,0,0.0,69.000,0,1,0.0,0.0,717.150
02-2486,1,64,1,2,4338.707,7.000,1089.000,588.688,276.264,827.419,...,2.037,4.746,0,0.0,49.948,0,0,0.0,0.0,274.183
06-0125,0,63,1,2,3621.526,6.000,1226.225,632.096,602.204,250.999,...,8.879,4.237,0,0.0,122.020,0,1,4.0,0.0,657.634


In [10]:
E.shape

(118,)

In [11]:
E[0:15]

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [12]:
E

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
Y.shape

(118,)

In [14]:
Y[0:15]

array([14.7189952,  2.4969724, 15.441803 , 16.1974657, 47.5738952,
       12.9448306, 11.8934738, 10.2835837,  7.1623682,  5.4210585,
       13.6019286,  1.0842117, 12.484862 ,  4.928235 ,  9.2650818])

In [15]:
gene_exp=gene_exp[['sex','Age','EGFR_re','CDKN2A_re']]

In [16]:
gene_exp.head()

,sex,Age,EGFR_re,CDKN2A_re
IID,,,,
02-0047,1,78,0,2
02-0055,0,62,1,0
02-2485,1,53,1,0
02-2486,1,64,1,2
06-0125,0,63,1,2


In [17]:
len(gene_exp.columns)

4

In [18]:
math.ceil(len(gene_exp.columns)**0.5)

2

In [19]:
math.ceil(len(gene_exp.columns)**0.5)**2

4


Add a number of columns to make the dataframe column count a square ( 236 x 236 = 55696 )


In [20]:
#for i in range(55696-len(gene_exp.columns)):
#    print("extra"+str(i), end=", ")
#    col=str("extra"+str(i))
#    gene_exp["extra"+str(i)]=0

extra=["extra"+str(i) for i in range( (math.ceil(len(gene_exp.columns)**0.5)**2) - len(gene_exp.columns) )]
#gene_exp[extra]=0

gene_exp=pd.concat([gene_exp, pd.DataFrame(columns = extra)]).fillna(0)

#gene_exp.assign(extra=0)
#gene_exp.insert(2, "expenditure", 4500, allow_duplicates=False) 

In [21]:
gene_exp.columns

Index(['sex', 'Age', 'EGFR_re', 'CDKN2A_re'], dtype='object')

In [22]:
len(gene_exp.columns)

4

In [23]:
X=np.array(gene_exp)

In [24]:
X.shape

(118, 4)

In [25]:
X[0:4,0:4]

array([[ 1, 78,  0,  2],
       [ 0, 62,  1,  0],
       [ 1, 53,  1,  0],
       [ 1, 64,  1,  2]])

In [26]:
X=X.astype('float64')

In [27]:
X.shape

(118, 4)

In [28]:
X[0:4,0:4]

array([[ 1., 78.,  0.,  2.],
       [ 0., 62.,  1.,  0.],
       [ 1., 53.,  1.,  0.],
       [ 1., 64.,  1.,  2.]])

In [29]:
# Remove the first two columns (OS and OS.time)
#X=X[:,2:]

In [30]:
X.shape

(118, 4)

In [31]:
X[0:4,0:4]

array([[ 1., 78.,  0.,  2.],
       [ 0., 62.,  1.,  0.],
       [ 1., 53.,  1.,  0.],
       [ 1., 64.,  1.,  2.]])

In [32]:
scaler=StandardScaler().fit(X)

In [33]:
X[0:4,0:4]

array([[ 1., 78.,  0.,  2.],
       [ 0., 62.,  1.,  0.],
       [ 1., 53.,  1.,  0.],
       [ 1., 64.,  1.,  2.]])

In [34]:
X=scaler.transform(X)

In [35]:
X[0:4,0:4]

array([[ 0.81361651,  1.22306379, -3.48010217,  0.50589175],
       [-1.22908027, -0.02108731,  0.28734789, -2.08955288],
       [ 0.81361651, -0.7209223 ,  0.28734789, -2.08955288],
       [ 0.81361651,  0.13443158,  0.28734789,  0.50589175]])

In [36]:
X.shape

(118, 4)

In [37]:
math.ceil(len(gene_exp.columns)**0.5)

2

In [38]:
score_tst_list = []
score_dev_list = []

#le, wi = 20, 5

# 236x236 is 55696

#le, wi = 10, 10
#le, wi = 236, 236
le, wi = math.ceil(len(gene_exp.columns)**0.5), math.ceil(len(gene_exp.columns)**0.5)

conv1_size=(1, wi)
conv2_size=(wi, 1)
input_shape = (le, wi, 1)

# using just 1 loop for now
for i in range(0,2):
        print("Range ")
        ci_tst_list = []
        ci_dev_list = []
        kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
        fold = 0
        for trn_index, tst_index in kf.split(X,E):
            print("Index ")
            fold +=1
            x_tst=X[tst_index]
            c_tst=E[tst_index]
            s_tst=Y[tst_index]

            x_trn, x_dev ,c_trn, c_dev , s_trn, s_dev=train_test_split(
                X[trn_index],E[trn_index],Y[trn_index],test_size=0.1,
                stratify=E[trn_index],random_state=1)

            sort_idx = np.argsort(s_trn)[::-1]
            x_trn = x_trn[sort_idx]
            s_trn = s_trn[sort_idx]
            c_trn = c_trn[sort_idx]

            x_trn = np.reshape(x_trn, (-1, le, wi))
            x_dev = np.reshape(x_dev, (-1, le, wi))
            x_tst = np.reshape(x_tst, (-1, le, wi))

            x_trn = x_trn.reshape(x_trn.shape[0], le, wi, 1)
            x_dev = x_dev.reshape(x_dev.shape[0], le, wi, 1)
            x_tst = x_tst.reshape(x_tst.shape[0], le, wi, 1)
        
            #fit
            data = (x_trn, c_trn, s_trn, x_dev, c_dev, s_dev)
            modelpath = save_path+"%s_fold_%s_repeat_%s_%s.hdf5" % (
                cancer_name,fold,i+1,le*wi)
            checkpoint = MyCallback(modelpath, data)
            
            
            

            
            
            conv1=128
            #conv2=8
            conv2=128
            #dense=32
            dense=64
            input_img = Input(input_shape)
            tower_1 = Conv2D(conv1, conv1_size, activation='relu')(input_img)
            tower_1 = MaxPooling2D(1, 2)(tower_1)
            tower_1 = Flatten()(tower_1)
            tower_2 = Conv2D(conv2, conv2_size, activation='relu')(input_img)
            tower_2 = MaxPooling2D(1, 2)(tower_2)
            tower_2 = Flatten()(tower_2)
            output = concatenate([tower_1, tower_2], axis=1)
            out1 = Dense(dense, activation='relu')(output)
            last_layer = Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(out1)
            model = Model(inputs=[input_img], outputs=last_layer)
            
            
            #conv1=128
            #dense=128
            #input_img = Input(input_shape)
            #tower = Conv2D(conv1, conv1_size, activation='relu')(input_img)
            #tower1 = MaxPooling2D(1, 2)(tower)
            #tower2 = Flatten()(tower1)
            #out = Dense(dense, activation='relu')(tower2)
            #last_layer = Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(out)
            #model = Model(inputs=[input_img], outputs=last_layer)

            #loss
            #def nll(E, NUM_E):
            #    def loss(y_true, y_pred):
            #        hazard_ratio = tf.squeeze(tf.exp(y_pred))
            #        log_risk = tf.math.log(tf.math.cumsum(hazard_ratio))
            #        uncensored_likelihood = tf.subtract(tf.squeeze(y_pred),log_risk)
            #        censored_likelihood = uncensored_likelihood * E
            #        neg_likelihood = -tf.reduce_sum(censored_likelihood) / NUM_E
            #        return neg_likelihood
            #    return loss
            
            model.compile(loss=[nll(c_trn, np.sum(c_trn))],optimizer="adam")
            #print(model.summary())
            
            
            print('Training...')
            history = model.fit(x_trn, s_trn, batch_size=len(x_trn),
                                epochs=100,verbose=0,callbacks=[checkpoint], shuffle=False)
            model.load_weights(modelpath)
        
            hr_pred=model.predict(x_dev,batch_size=1, verbose=1)
            hr_pred=np.exp(hr_pred)
            #print(hr_pred, c_dev, s_dev)
            print(pd.DataFrame({ 
                'hr_pred': [item for row in hr_pred for item in row], 's_dev': s_dev, 'c_dev': c_dev }))

            
            #fig,ax = plt.subplots(1)
            #ax.scatter(hr_pred,s_dev)
            ##log_pred = [-np.log(p) for p in hr_pred]
            ##ax.scatter(hr_pred,log_pred)
            #plt.title("2D - hr_pred vs s_dev")
            #plt.show()
            
            ci_dev=concordance_index(s_dev,-hr_pred,c_dev)
            
            hr_pred2=model.predict(x_tst,batch_size=1, verbose=1)
            hr_pred2=np.exp(hr_pred2)
            #print(hr_pred2, c_tst, s_tst)
            
            print(pd.DataFrame({ 
                'hr_pred2': [item for row in hr_pred2 for item in row], 's_tst': s_tst, 'c_tst': c_tst }))
            
            #fig,ax = plt.subplots(1)
            #ax.scatter(hr_pred2,s_tst)
            #plt.title("2D - hr_pred2 vs s_tst")
            #plt.show()
            
            ci_tst=concordance_index(s_tst,-hr_pred2,c_tst)
            
            ci_dev_list.append(ci_dev)
            ci_tst_list.append(ci_tst)
            
        score_dev_list.append(ci_dev_list)
        score_tst_list.append(ci_tst_list)


Range 
Index 
Training...
Early stopping at 21
[Best:on_train_end]

=========================== [Best cindex (epoch = 0)] cindex=0.666667 ==================================
10/10 [==============================] - 0s 1ms/step
    hr_pred      s_dev  c_dev
0  0.998644   6.143866      0
1  0.999972  14.718995      1
2  1.000505   6.800964      0
3  1.000877   2.234133      1
4  1.000593  10.612133      1
5  1.000566   8.969388      0
6  1.001626  12.484862      1
7  1.000802   4.533976      1
8  0.998335   0.427114      0
9  1.000078   2.496972      1
24/24 [==============================] - 0s 966us/step
    hr_pred2      s_tst  c_tst
0   1.000695   4.369702      1
1   1.000421   8.345145      1
2   1.000536   8.542274      0
3   1.000627   4.566831      0
4   0.998211   4.698251      0
5   1.001930   1.872729      1
6   0.999663  17.840211      1
7   1.000461   7.162368      0
8   1.000421   4.632541      1
9   1.000957   4.862525      1
10  1.000276   6.768109      0
11  1.000802   3.

Training...
Early stopping at 21
[Best:on_train_end]

=========================== [Best cindex (epoch = 0)] cindex=0.760870 ==================================
10/10 [==============================] - 0s 3ms/step
    hr_pred      s_dev  c_dev
0  0.997385   4.698251      0
1  1.000834   2.496972      1
2  0.999505   8.805113      0
3  0.998554  11.236376      1
4  0.998032  10.940682      1
5  0.999199   4.566831      0
6  0.999334   4.369702      1
7  1.001353   9.166517      1
8  0.999004   8.542274      0
9  0.998554  12.944831      1
24/24 [==============================] - 0s 3ms/step
    hr_pred2      s_tst  c_tst
0   0.999201  14.718995      1
1   1.000893  47.573895      1
2   0.997030   5.421059      0
3   1.000457   2.825521      0
4   0.999076   8.969388      0
5   1.001721   2.726957      1
6   1.000337   2.562682      1
7   1.001655  28.912312      1
8   1.000012  21.881363      1
9   1.000893  25.363983      1
10  1.000062   9.396501      0
11  1.000687   9.856470      1
12


If the hazard ratio is less than 1, then the predictor is protective (i.e., associated with improved survival) and if the hazard ratio is greater than 1, then the predictor is associated with increased risk (or decreased survival).
